## Self-oscillating sinusoidal circuits
### Phase-shift sinusoidal oscillator

![Alt Text](https://www.edutecnica.it/elettronica/oscillatori/23.png)

The figure above shows a phase-shift sinusoidal oscillator: the action block is implemented with an Op-amp in an inverting configuration, while the reaction block, also known as the feedback block, is realized with a network of resistors and capacitors, ensuring a 180° phase shift between the input and output signals. To determine the amplification factor and the frequency of the self-generated sinusoidal signal, Berkhausen's criterion is applied as shown:


In [1]:
# Importing SymPy
import sympy as smp

In [2]:
# Defining the solver function
def solver(M, b):
    
    inv = M.inv()
    sol = inv * b

    return sol


# Defining the berk function
def berk(AB, A, w):
    
    eq1 = smp.Eq(smp.simplify(smp.Abs(AB)), 1)
    eq2 = smp.Eq(smp.simplify(smp.atan(smp.im(AB) / smp.re(AB))), 0)
    
    sol = smp.solve([eq1, eq2], (A, w))[0]

    return sol

In [3]:
# Defining symbols
Z = smp.symbols("Z", imaginary = True)
R = smp.symbols("R", real = True, positive = True, constant = True)
C = smp.symbols("C", real = True, positive = True, constant = True)
V = smp.symbols("V", complex = True, nonzero = True)
A = smp.symbols("A", real = True, positive = True)
w = smp.symbols("omega", real = True, positive = True)

The first step is to calculate the gain of the feedback network. To do this, it is necessary to apply Kirchhoff's two laws, linearize the relationship between current and voltage for each capacitor using the phasor method, write the resulting linear system in matrix form, and finally solve it.

In [4]:
# Coefficient matrix of the linear system
M = smp.Matrix([[Z + R, Z, Z], [-R, Z + R, Z], [0, -R, Z + R]])
M

Matrix([
[R + Z,     Z,     Z],
[   -R, R + Z,     Z],
[    0,    -R, R + Z]])

In [5]:
# Right-hand side vector
b = smp.Matrix([V, 0, 0])

In [6]:
# Getting the currents
sol = solver(M, b)
sol

Matrix([
[V*(R**2 + 3*R*Z + Z**2)/(R**3 + 6*R**2*Z + 5*R*Z**2 + Z**3)],
[         V*(R**2 + R*Z)/(R**3 + 6*R**2*Z + 5*R*Z**2 + Z**3)],
[                 R**2*V/(R**3 + 6*R**2*Z + 5*R*Z**2 + Z**3)]])

In [7]:
# Defining B
B = sol[2] * R / V

# Capacitor impedance
Z_C = 1 / (smp.I * w * C)

# Defining AB
AB = - A * smp.simplify(B.subs(Z, Z_C))
AB

-A*C**3*R**3*omega**3/(C**3*R**3*omega**3 - 6*I*C**2*R**2*omega**2 - 5*C*R*omega + I)

After finding the gain of the feedback network, it is multiplied by the gain of the action block to obtain the complex function $AB$. The next step is to calculate the magnitude of $AB$ and set it equal to 1, and impose that the phase of $AB$ is zero as well. The solutions to both equations are the gain of the action block and the frequency of the self-generated sinusoidal wave.

\begin{equation*}
\begin{cases}
| AB | = 1 \\
\angle \left( AB \right) = 0
\end{cases}
\end{equation*}

In [8]:
# Getting A and omega
A0, w0 = berk(AB, A, w)

In [9]:
A0

29

In [10]:
w0

sqrt(6)/(6*C*R)

###  Generalized Wien bridge oscillator

![Alt Text](https://www.edutecnica.it/elettronica/oscillatori/15.png)

We are also going to solve the Wien bridge oscillator, as we did before. However, we are determined to find more general formulas.

In [11]:
Z1, Z2 = smp.symbols("Z_1, Z_2", imaginary = True)
R1, R2 = smp.symbols("R_1, R_2", real = True, positive = True, constant = True)
C1, C2 = smp.symbols("C_1, C_2", real = True, positive = True, constant = True)

In [12]:
M = smp.Matrix([[Z1 + Z2 + R1, -Z2], [-Z2, R2 + Z2]])
M

Matrix([
[R_1 + Z_1 + Z_2,      -Z_2],
[           -Z_2, R_2 + Z_2]])

In [13]:
b = smp.Matrix([V, 0])

In [14]:
sol = solver(M, b)
sol

Matrix([
[V*(R_2 + Z_2)/(R_1*R_2 + R_1*Z_2 + R_2*Z_1 + R_2*Z_2 + Z_1*Z_2)],
[        V*Z_2/(R_1*R_2 + R_1*Z_2 + R_2*Z_1 + R_2*Z_2 + Z_1*Z_2)]])

In [15]:
B = sol[1] * R2 / V

Z_C1 = 1 / (smp.I * w * C1)
Z_C2 = 1 / (smp.I * w * C2)

AB = A * smp.simplify(B.subs([(Z1, Z_C1), (Z2, Z_C2)]))
AB

I*A*C_1*R_2*omega/(-C_1*C_2*R_1*R_2*omega**2 + I*C_1*omega*(R_1 + R_2) + I*C_2*R_2*omega + 1)

In [16]:
A0, w0 = berk(AB, A, w)

In [17]:
A0.factor().expand()

R_1/R_2 + 1 + C_2/C_1

In [18]:
w0

1/(sqrt(C_1)*sqrt(C_2)*sqrt(R_1)*sqrt(R_2))

It is very interesting to note that, in the general case, the amplifier’s gain depends on the circuit components of the feedback network. By imposing that all components have the same value, the known results are obtained:

In [19]:
A0.subs([(R1, R), (R2, R), (C1, C), (C2, C)])

3

In [20]:
w0.subs([(R1, R), (R2, R), (C1, C), (C2, C)])

1/(C*R)